Encoder   
Decoder   
LSTMAutoEncoder

In [ ]:
import torch
import torch.nn as nn

"""
LSTM output
- N : number of batches
- L : sequence lengh
- Q : input dim
- K : number of layers
- D : LSTM feature dimension

Y,(hn,cn) = LSTM(X)

- X : [N x L x Q] - `N` input sequnce of length `L` with `Q` dim.
- Y : [N x L x D] - `N` output sequnce of length `L` with `D` feature dim.
- hn : [K x N x D] - `K` (per each layer) of `N` final hidden state with  `D` feature dim.
- cn : [K x N x D] - `K` (per each layer) of `N` final hidden state with  `D` cell dim.
"""


class Encoder(nn.Module):
    """
    input: input_seq: (batch_size, seq_len, n_features) -> (1, 20, 38)
    output: hidden_cell -> (hn, cn)
        -> ((num_layers, batch_size, hidden_size), (num_layers, batch_size, hidden_size))
    """

    def __init__(self, num_layers, hidden_size, n_features, device):
        super(Encoder, self).__init__()

        self.input_size = n_features
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device

        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )

    def initHidden(self, batch_size):
        """
        intialize hn, cn
        """
        self.hidden_cell = (
            torch.randn(
                (self.num_layers, batch_size, self.hidden_size), dtype=torch.float
            ).to(self.device),
            torch.randn(
                (self.num_layers, batch_size, self.hidden_size), dtype=torch.float
            ).to(self.device),
        )

    def forward(self, input_seq):
        self.initHidden(input_seq.shape[0])
        _, self.hidden_cell = self.lstm(input_seq, self.hidden_cell)
        return self.hidden_cell


class Decoder(nn.Module):
    """
    input: (input_seq, hidden_cell)
        input_seq:
        hidden_cell: encoder 에서 넘어온 hidden_cell (hn, cn)
    output:
        decoder output: (batch_size, seq_len, n_features) -> (1, 1, 38)
        linear output: (batch_size, n_features) -> (1, 38)
    """

    def __init__(self, num_layers, hidden_size, n_features, device):
        super(Decoder, self).__init__()

        self.input_size = n_features
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device

        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )
        self.linear = nn.Linear(in_features=hidden_size, out_features=n_features)

    def forward(self, input_seq, hidden_cell):
        output, hidden_cell = self.lstm(input_seq, hidden_cell)
        output = self.linear(output)
        return output, hidden_cell


class LSTMAutoEncoder(nn.Module):
    """
    output: input seq_len(20) 모두 복원
        reconstruction 순서는 입력의 반대.
    """

    def __init__(self, num_layers, hidden_size, n_features, device):
        super(LSTMAutoEncoder, self).__init__()
        self.device = device
        self.encoder = Encoder(num_layers, hidden_size, n_features, device)
        self.decoder = Decoder(num_layers, hidden_size, n_features, device)

    def forward(self, input_seq):
        output = torch.zeros(size=input_seq.shape, dtype=torch.float)
        hidden_cell = self.encoder(input_seq)
        input_decoder = torch.zeros(
            (input_seq.shape[0], 1, input_seq.shape[2]), dtype=torch.float
        ).to(self.device)
        for i in range(input_seq.shape[1] - 1, -1, -1):
            output_decoder, hidden_cell = self.decoder(input_decoder, hidden_cell)
            input_decoder = output_decoder
            output[:, i, :] = output_decoder[:, 0, :]

        return output.to(self.device)

In [ ]:
import json
import os
import os.path as osp
from collections import defaultdict as dd

import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class AbnormalDataset(Dataset):
    def __init__(
        self,
        sequence_length=20,
        root="/content/drive/MyDrive/taba/유원/npy_from_MAEv2/절도 영상 원천 데이터",
        label_root="/data/ephemeral/home/level2-3-cv-finalproject-cv-06/app/models/lstmae/dataset/label",
    ):
        super().__init__()
        self.sequence_length = sequence_length

        self.scaler = MinMaxScaler()
        # 데이터 값 [0,1] 범위로 scaling할때 사용

        # Load the dataset
        file_list = os.listdir(root)

        df_list = []
        self.length = 0
        self.range_table = []
        self.real_length = 0
        self.real_idx_table = []

        for i, file_name in enumerate(file_list):
            dat = pd.read_csv(root + "/" + file_name)
            # dat.drop(columns=["Frame"], inplace=True)  # Remove the 'Frame' column

            print(f"==>>{i}번째 dat.shape: {dat.shape}")

            id_counter = pd.Series(dat["ID"]).value_counts(sort=False)

            for id_to_del in id_counter[id_counter < sequence_length].index:
                dat.drop(dat[dat["ID"] == id_to_del].index, inplace=True)

            id_counter = pd.Series(dat["ID"]).value_counts(sort=False)

            print(f"==>>{i}번째 처리 후 dat.shape: {dat.shape}")
            assert len(id_counter[id_counter < sequence_length].index) == 0

            for count in id_counter:
                cur_id_length = count - sequence_length + 1
                self.range_table.append(self.length + cur_id_length)
                self.real_idx_table.append(self.real_length + count)
                self.length += cur_id_length
                self.real_length += count

            dat["ID"] = dat["ID"].astype("str") + f"_{i}"
            df_list.append(dat.copy())

        self.dat = pd.concat(df_list, ignore_index=True)

        # 정답 frame 담은 dict 만들기
        self.frame_label = dd(lambda: dd(lambda: [-1, -1]))

        folder_list = os.listdir(label_root)

        for folder in folder_list:
            json_list = os.listdir(label_root + "/" + folder)

            for js in json_list:
                with open(label_root + "/" + folder + "/" + js, "r") as j:
                    json_dict = json.load(j)

                for dict in json_dict["annotations"]["track"]:
                    if dict["@label"].endswith("_start"):
                        cur_id = dict["@id"]
                        self.frame_label[js[:-5]][cur_id][0] = dict["box"][0]["@frame"]
                    elif dict["@label"].endswith("_end"):
                        cur_id = dict["@id"]
                        self.frame_label[js[:-5]][cur_id][1] = dict["box"][0]["@frame"]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        real_idx = self.find_real_idx(idx)

        sequence = self.dat[real_idx : real_idx + self.sequence_length].copy()
        target_frames = sequence["Frame"].values
        target_filename = sequence["Filename"].unique()[0].split(".")[0]
        sequence.drop(columns=["ID"], inplace=True)
        sequence.drop(columns=["Frame"], inplace=True)
        sequence.drop(columns=["Filename"], inplace=True)
        # sequence = self.scaler.fit_transform(sequence.values)
        sequence = np.array(sequence)

        target_labels = []

        for target_frame in target_frames:
            temp = 0
            for cur_id in range(0, len(self.frame_label[target_filename].keys()), 2):
                if int(target_frame) >= int(
                    self.frame_label[target_filename][str(int(cur_id))][0]
                ) and int(target_frame) <= int(
                    self.frame_label[target_filename][str(int(cur_id) + 1)][1]
                ):
                    temp = 1

            target_labels.append(temp)

        target_labels = torch.LongTensor(target_labels)

        return (sequence, target_labels)

    def find_real_idx(self, idx):

        start = 0
        end = len(self.range_table) - 1
        while start <= end:
            mid = (start + end) // 2
            if self.range_table[mid] == idx:
                real_idx = idx + ((mid + 1) * (self.sequence_length - 1))
                return real_idx

            if self.range_table[mid] > idx:
                end = mid - 1
            else:
                start = mid + 1

        real_idx = idx + (start * (self.sequence_length - 1))

        return real_idx

NameError: name 'Dataset' is not defined

In [ ]:
import os
import os.path as osp
import random
from argparse import ArgumentParser
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from dataset import NormalDataset

from lstm_ae_old import LSTMAutoencoder
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm


def parse_args():
    parser = ArgumentParser()

    # 학습 데이터 경로
    parser.add_argument(
        "--root_dir",
        type=str,
        default=os.environ.get(
            "SM_CHANNEL_TRAIN_CSV",
            "/data/ephemeral/home/level2-3-cv-finalproject-cv-06/app/models/lstmae/dataset/normal",
        ),
    )

    # pth 파일 저장 경로
    parser.add_argument(
        "--model_dir",
        type=str,
        default=os.environ.get(
            "SM_MODEL_DIR",
            "/data/ephemeral/home/level2-3-cv-finalproject-cv-06/app/models/pts",
        ),
    )

    # import_module로 불러올 model name
    parser.add_argument("--model_name", type=str, default="LSTM")
    # resume 파일 이름
    parser.add_argument("--resume_name", type=str, default="")
    # random seed
    parser.add_argument("--seed", type=int, default=666)

    parser.add_argument(
        "--device", default="cuda" if torch.cuda.is_available() else "cpu"
    )
    parser.add_argument("--num_workers", type=int, default=8)
    parser.add_argument("--batch_size", type=int, default=64)
    parser.add_argument("--val_batch_size", type=int, default=64)
    parser.add_argument("--val_num_workers", type=int, default=8)
    parser.add_argument("--learning_rate", type=float, default=0.001)
    parser.add_argument("--max_epoch", type=int, default=50)

    parser.add_argument("--save_interval", type=int, default=1)
    parser.add_argument("--val_interval", type=int, default=1)
    parser.add_argument("--thr", type=float, default=0.02)

    parser.add_argument("--patience", type=int, default=10)

    parser.add_argument("--wandb_mode", type=str, default="online")
    parser.add_argument("--wandb_run_name", type=str, default="LSTM-AE")

    args = parser.parse_args()

    return args


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


def train(
    root_dir,
    model_dir,
    model_name,
    device,
    num_workers,
    batch_size,
    val_num_workers,
    val_batch_size,
    learning_rate,
    max_epoch,
    val_interval,
    save_interval,
    thr,
    patience,
    resume_name,
    seed,
    wandb_mode,
    wandb_run_name,
):

    time_start = datetime.now()

    train_start = time_start.strftime("%Y%m%d_%H%M%S")

    set_seed(seed)

    if not osp.exists(model_dir):
        os.makedirs(model_dir)

    # Define parameters
    sequence_length = 20
    prediction_time = 1
    n_features = 38

    batch_size = batch_size
    val_batch_size = val_batch_size

    # -- early stopping flag
    patience = patience
    counter = 0

    # 데이터셋
    dataset = NormalDataset(
        root=root_dir,
    )

    valid_data_size = len(dataset) // 10

    train_data_size = len(dataset) - valid_data_size

    train_dataset, valid_dataset = random_split(
        dataset, lengths=[train_data_size, valid_data_size]
    )

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    valid_loader = DataLoader(
        dataset=valid_dataset,
        batch_size=val_batch_size,
        shuffle=False,
        num_workers=val_num_workers,
    )

    data_load_end = datetime.now()
    data_load_time = data_load_end - time_start
    data_load_time = str(data_load_time).split(".")[0]
    print(f"==>> data_load_time: {data_load_time}")

    # Initialize the LSTM autoencoder model
    # model = LSTMAutoencoder(sequence_length, prediction_time, n_features, 50)
    # model.to(device)

    model = LSTMAutoEncoder(
        num_layers=2, hidden_size=50, n_features=n_features, device=device
    )
    model.to(device)

    optimizer = torch.optim.Adam(
        model.parameters(), lr=learning_rate, weight_decay=1e-6
    )

    scheduler = torch.optim.lr_scheduler.MultiStepLR(
        optimizer, milestones=[15, 40], gamma=0.1
    )

    # if resume_name:
    #     optimizer.load_state_dict(load_dict["optimizer_state_dict"])
    #     scheduler.load_state_dict(load_dict["scheduler_state_dict"])
    #     scaler.load_state_dict(load_dict["scaler_state_dict"])

    criterion = nn.MSELoss()
    val_criterion = nn.MSELoss(reduction="none")

    print(f"Start training..")

    wandb.init(
        project="VAD",
        config={
            "lr": learning_rate,
            "dataset": "무인매장",
            "n_epochs": max_epoch,
            "loss": "MSE",
            "notes": "VAD 실험",
        },
        name=wandb_run_name + "_" + train_start,
        mode=wandb_mode,
    )

    wandb.watch((model,))

    best_loss = np.inf
    total_batches = len(train_loader)

    for epoch in range(max_epoch):
        model.train()

        epoch_start = datetime.now()

        epoch_loss = 0

        for step, data in tqdm(enumerate(train_loader), total=total_batches):

            data = data.to(device)
            optimizer.zero_grad()

            pred = model(data)

            loss = criterion(pred, data)

            loss.backward()
            optimizer.step()

            epoch_loss += loss

        epoch_mean_loss = (epoch_loss / total_batches).item()

        train_end = datetime.now()
        train_time = train_end - epoch_start
        train_time = str(train_time).split(".")[0]
        print(
            f"==>> epoch {epoch+1} train_time: {train_time}\nloss: {round(epoch_mean_loss,4)}"
        )

        if (epoch + 1) % save_interval == 0:

            ckpt_fpath = osp.join(model_dir, f"{model_name}_{train_start}_latest.pth")

            states = {
                "epoch": epoch,
                "model_name": model_name,
                "model_state_dict": model.state_dict(),  # 모델의 state_dict 저장
                "optimizer_state_dict": optimizer.state_dict(),
                "scheduler_state_dict": scheduler.state_dict(),
                # "scaler_state_dict": scaler.state_dict(),
            }

            torch.save(states, ckpt_fpath)

        # validation 주기에 따라 loss를 출력하고 best model을 저장합니다.
        if (epoch + 1) % val_interval == 0:

            print(f"Start validation #{epoch+1:2d}")
            model.eval()

            with torch.no_grad():
                total_loss = 0

                for step, data in tqdm(
                    enumerate(valid_loader), total=len(valid_loader)
                ):

                    data = data.to(device)

                    pred = model(data)

                    val_loss = val_criterion(pred, data)
                    val_loss = torch.mean(val_loss)

                    total_loss += val_loss

                val_mean_loss = (total_loss / len(valid_loader)).item()

            if best_loss > val_mean_loss:
                print(
                    f"Best performance at epoch: {epoch + 1}, {best_loss:.4f} -> {val_mean_loss:.4f}"
                )
                print(f"Save model in {model_dir}")
                states = {
                    "epoch": epoch,
                    "model_name": model_name,
                    "model_state_dict": model.state_dict(),  # 모델의 state_dict 저장
                    # "optimizer_state_dict": optimizer.state_dict(),
                    # "scheduler_state_dict": scheduler.state_dict(),
                    # "scaler_state_dict": scaler.state_dict(),
                    # best.pth는 inference에서만 쓰기?
                }

                best_ckpt_fpath = osp.join(
                    model_dir, f"{model_name}_{train_start}_best.pth"
                )
                torch.save(states, best_ckpt_fpath)
                best_loss = val_mean_loss
                counter = 0
            else:
                counter += 1

        new_wandb_metric_dict = {
            "train_loss": epoch_mean_loss,
            "valid_loss": val_mean_loss,
            "learning_rate": scheduler.get_lr()[0],
        }

        wandb.log(new_wandb_metric_dict)

        scheduler.step()

        epoch_end = datetime.now()
        epoch_time = epoch_end - epoch_start
        epoch_time = str(epoch_time).split(".")[0]
        print(
            f"==>> epoch {epoch+1} time: {epoch_time}\nvalid_loss: {round(val_mean_loss,4)}"
        )

        if counter > patience:
            print("Early Stopping...")
            break

    time_end = datetime.now()
    total_time = time_end - time_start
    total_time = str(total_time).split(".")[0]
    print(f"==>> total time: {total_time}")


def main(args):
    train(**args.__dict__)


if __name__ == "__main__":
    args = parse_args()

    main(args)